In [1]:
def start_transcribe_job(transcribe, job_name, bucket, file):
    """Starts an AWS Transcribe job
    
    Parameters
    ----------
    transcribe : AWS `transcribe` service client instance
    job_name : str, name of the AWS job
    bucket : str, AWS S3 bucket name
    file : str, name of the mp3 file to transcribe
    
    Returns
    -------
    True: if job started successfully
    
    """
    # this is the location of your mp3 file
    file_uri = f'https://s3.amazonaws.com/{bucket}/{file}'
    
    # try launching the job and return any error messages
    try:
        transcribe.start_transcription_job(
            TranscriptionJobName=job_name,
            Media={'MediaFileUri': file_uri},
            MediaFormat='mp3',
            LanguageCode='en-US')
        return True
    except Exception as e:
        return e

In [5]:
def get_transcription_text(transcribe, job_name):
    """Returns transcription text for the AWS Transcribe job
    
    Parameters
    ----------
    transcribe : AWS `transcribe` service client instance
    job_name : transcibe service `job` name
    
    Returns
    -------
    Current job status if job is still in progress
    Transcription text if job is Completed
    """
    import urllib.request
    import json
    import time
    
    # let's obtain the job instance
    job = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    
    # and it's status
    status = job['TranscriptionJob']['TranscriptionJobStatus']
    
    # check the status every 5 seconds and 
    # return the transcribed text if the job is finished
    # otherwise return None if job failed
    while True:
        if status == 'COMPLETED':
            print(f"Job {job_name} completed")
            with urllib.request.urlopen(job['TranscriptionJob']['Transcript']['TranscriptFileUri']) as r:
                data = json.loads(r.read())
            return data['results']['transcripts'][0]['transcript']
        elif status == 'FAILED':
            print(f"Job {job_name} failed")
            return None
        else:
            print(f"Status of job {job_name}: {status}")
            time.sleep(5)

In [7]:
# 'rootkey.csv' is the file you've downloaded from AWS
with open('rootkey.csv', 'r') as f:
    content = f.readlines()

keys = {}
for line in content:
    pair = line.strip().split('=')
    keys.update({pair[0] : pair[1]})

AWS_ACCESS_KEY_ID = keys['AWSAccessKeyId']
AWS_SECRET_KEY = keys['AWSSecretKey']

In [ ]:
import boto3

BUCKET_NAME = 'springboard-test'
FILE_NAME = 'testing.mp3'
JOB_NAME = 'testing_transcribe'

# Instantiate a client to the AWS transcribe service
transcribe = boto3.client(
    'transcribe',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name='us-east-1')

In [9]:
# launch the job
job_status = start_transcribe_job(transcribe, JOB_NAME, BUCKET_NAME, FILE_NAME)

# if job launched successfully `job_status` will be True
if job_status: # and we can start requesting the results from the service
    text = get_transcription_text(transcribe, JOB_NAME)
    print(f'The transcribed text for {FILE_NAME} file:')
    print(text)
else: # or print the error code if somethign went wrong
    print(f'Job {JOB_NAME} failed with the error: {job_status}')

Job testing_transcribe completed
The transcribed text for testing.mp3 file:
Chancellor. Mr. Vice Chancellor Professor Robertson Diamond Daniel that, ladies and gentlemen, I come here evening because of my deep interest and affection for a land settled by the Dutch in the mid 17 century, then taken over by the British and it last independent, a land in which the native inhabitants, where first subdued but relations with whom remain a problem to this day, a land which defined itself on a hostile frontier. A land which has attained rich natural resource is through the energetic application of modern technology, a land which was once the importer of play and now must struggled to wipe out the last traces of that form of bondage. I refer cost to the United States of America. But I'm glad to come here. My wife and I and all of our party. You're glad to come here to South Africa, and we're glad to come here to Cape Town. I am already greatly enjoying. I stay in my visit here. I am making an e